In [1]:
from dtw_soft import *

In [2]:
x = torch.tensor([2.0,5.0,3.0,6.0],requires_grad=True).unsqueeze(-1)
y = torch.tensor([5.0,9.0,2.0],requires_grad=True).unsqueeze(-1)

loss = soft_dtw(x,y)[0]


In [3]:
loss.backward()

In [4]:
x1 = torch.tensor([2.0,5.0,3.0,6.0],requires_grad=True).unsqueeze(-1)
x2 = torch.tensor([2.0,3.0,3.0,5.0],requires_grad=True).unsqueeze(-1)
x = torch.stack([x1,x2])

y1 = torch.tensor([5.0,9.0,2.0,2.0],requires_grad=True).unsqueeze(-1)
y2 = torch.tensor([2.0,3.0,3.0,2.0],requires_grad=True).unsqueeze(-1)
y = torch.stack([y1,y2])

print(x.shape,y.shape)

loss = torch.mean(soft_dtw_batch_same_size(x,y)[0])

torch.Size([2, 4, 1]) torch.Size([2, 4, 1])


In [5]:
loss

tensor(23.8546, grad_fn=<MeanBackward0>)

In [6]:
loss.backward()

In [7]:
str(0.000000005)

'5e-09'

In [8]:
import subprocess

# List all large files in your repository
subprocess.run("git lfs migrate info --everything", shell=True)

CompletedProcess(args='git lfs migrate info --everything', returncode=0)

In [9]:
import torch

# Creating two 5-length vectors
x = torch.randn(5, requires_grad=True)
y = torch.randn(5, requires_grad=True)

# Computing L2 norm between x and y
l2_norm = torch.norm(x - y, p=2)

# Computing the gradient
l2_norm.backward()

# Extracting gradients
x_grad = x.grad
y_grad = y.grad

x_grad, y_grad

(tensor([-0.4135, -0.4927,  0.3583,  0.6765,  0.0147]),
 tensor([ 0.4135,  0.4927, -0.3583, -0.6765, -0.0147]))

In [10]:
x_grad_theoretical = (x - y) / l2_norm
y_grad_theoretical = (y - x) / l2_norm

x_grad_theoretical, y_grad_theoretical

(tensor([-0.4135, -0.4927,  0.3583,  0.6765,  0.0147], grad_fn=<DivBackward0>),
 tensor([ 0.4135,  0.4927, -0.3583, -0.6765, -0.0147], grad_fn=<DivBackward0>))

In [23]:
x = torch.tensor([2.0,5.0,3.0,6.0],requires_grad=True).unsqueeze(-1)
y = torch.tensor([5.0,9.0,2.0],requires_grad=True).unsqueeze(-1)

# Retaining the gradient for non-leaf tensor
x.retain_grad()
y.retain_grad()

loss = soft_dtw(x,y)[0]

In [12]:
loss.backward()

# Extracting gradients
x_grad = x.grad
y_grad = y.grad

x_grad, y_grad

(tensor([[0.],
         [0.],
         [0.],
         [8.]]),
 tensor([[ 0.],
         [ 0.],
         [-8.]]))

In [26]:
E = backward_recursion(x,y)

In [27]:
E

tensor([[1.0000e+00, 1.5002e-20, 7.4977e-21],
        [9.8696e-01, 2.6084e-02, 3.2187e-06],
        [9.7392e-01, 1.0178e-10, 2.6084e-02],
        [2.6193e-01, 9.7392e-01, 1.0000e+00]], grad_fn=<SliceBackward0>)

In [15]:
print(x.repeat(3,1,1).shape)

E = backward_recursion_batch_same_size(x.repeat(3,1,1),y.repeat(3,1,1))



torch.Size([3, 4, 1])


In [16]:
E

tensor([[[1.0000e+00, 1.5002e-20, 7.4977e-21],
         [9.8696e-01, 2.6084e-02, 3.2187e-06],
         [9.7392e-01, 1.0178e-10, 2.6084e-02],
         [2.6193e-01, 9.7392e-01, 1.0000e+00]],

        [[1.0000e+00, 1.5002e-20, 7.4977e-21],
         [9.8696e-01, 2.6084e-02, 3.2187e-06],
         [9.7392e-01, 1.0178e-10, 2.6084e-02],
         [2.6193e-01, 9.7392e-01, 1.0000e+00]],

        [[1.0000e+00, 1.5002e-20, 7.4977e-21],
         [9.8696e-01, 2.6084e-02, 3.2187e-06],
         [9.7392e-01, 1.0178e-10, 2.6084e-02],
         [2.6193e-01, 9.7392e-01, 1.0000e+00]]], grad_fn=<SliceBackward0>)

In [29]:
def back(x,y,E):
    p = len(x[0])
    m = len(y)
    print(p,m)

    a = (torch.matmul(torch.ones(p,m), E.transpose(0,1)))
    a= a * x
    b = (y.transpose(0,1) @ E.transpose(0,1))
    return 2*(a-b)

In [30]:
print(x.shape)

back(x,y,E)

torch.Size([4, 1])
1 3


tensor([[ -6.0000,  -6.2869,  -5.8435, -15.2064],
        [  0.0000,  -0.2087,   0.1565,  -1.7913],
        [ -4.0000,  -4.2608,  -3.8435, -10.7347],
        [  2.0000,   1.8174,   2.1565,   2.6803]], grad_fn=<MulBackward0>)